<a href="https://colab.research.google.com/github/ibacaraujo/to-tensorflow2/blob/master/classify_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classify text

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

!pip install -q tensorflow==2.0.0-beta1
import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print('Version: ', tf.__version__)
print('Eager mode: ', tf.executing_eagerly())
print('Hub version: ', hub.__version__)
print('GPU is', 'available' if tf.test.is_gpu_available() else 'NOT AVAILABLE')

     |████████████████████████████████| 87.9MB 1.3MB/s 
     |████████████████████████████████| 3.1MB 43.6MB/s 
     |████████████████████████████████| 501kB 47.1MB/s 
Version:  2.0.0-beta1
Eager mode:  True
Hub version:  0.5.0
GPU is NOT AVAILABLE


## The dataset, the IMDB dataset

In [4]:
# first define the splits
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])
test_split = tfds.Split.TEST

# download considering the split
# 15,000 examples for training, 10,000 examples for validation
# 25,000 examples for test
(train_data, validation_data), test_data = tfds.load(
    name='imdb_reviews',
    split=(train_validation_split, test_split),
    as_supervised=True
)

W0803 19:21:12.071883 139630417131392 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_datasets/core/file_format_adapter.py:209: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


## Explore the data

A moment to understand the format of the data.

Each example is a sentence reprensenting the movie review and a corresponding label. The sentence is not processed in any way. The label is an integer value, 0 is a negative review, 1 is a positive review.

In [5]:
# Print first 10 examples
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: id=220, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

In [6]:
train_labels_batch

<tf.Tensor: id=221, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

## Build the model!

Three main architectural decisions for dealing with text.

1. How to represent the text?
2. How many layers to use in the model?
3. How many hidden units to use for each layer?

In [13]:
# embedding vector to represent text with pretrained models
embedding = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string,
                           trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=12433, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [14]:
# the full model
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Train the model!

In [16]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs=20, 
                    validation_data=validation_data.batch(512), verbose=1)

Epoch 1/20
30/30 [==============================] - 6s 212ms/step - loss: 0.6683 - accuracy: 0.6005 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 6s 185ms/step - loss: 0.6098 - accuracy: 0.6716 - val_loss: 0.5924 - val_accuracy: 0.6895
Epoch 3/20
30/30 [==============================] - 6s 186ms/step - loss: 0.5647 - accuracy: 0.7187 - val_loss: 0.5541 - val_accuracy: 0.7216
Epoch 4/20
30/30 [==============================] - 5s 183ms/step - loss: 0.5242 - accuracy: 0.7528 - val_loss: 0.5197 - val_accuracy: 0.7505
Epoch 5/20
30/30 [==============================] - 6s 185ms/step - loss: 0.4858 - accuracy: 0.7840 - val_loss: 0.4874 - val_accuracy: 0.7742
Epoch 6/20
30/30 [==============================] - 6s 185ms/step - loss: 0.4489 - accuracy: 0.8085 - val_loss: 0.4570 - val_accuracy: 0.7922
Epoch 7/20
30/30 [==============================] - 6s 184ms/step - loss: 0.4139 - accuracy: 0.8293 - val_loss: 0.4292 - val_accuracy: 0.808

## Evaluate the model

In [17]:
results = model.evaluate(test_data.batch(512), verbose=0)
for name, value in zip(model.metrics_names, results):
  print('%s: %.3f' % (name, value))

loss: 0.323
accuracy: 0.862


## Further exploration

Let me see how different embedding model impact results.

In [7]:
embedding2 = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1"
hub_layer = hub.KerasLayer(embedding2, input_shape=[], dtype=tf.string,
                           trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=406, shape=(3, 50), dtype=float32, numpy=
array([[ 1.4732717 ,  0.42393148,  0.16215092,  0.5873862 ,  0.1513372 ,
        -0.5062546 , -0.21033262, -0.33339503, -0.7381036 ,  0.29208365,
        -0.35330555, -0.02719353, -0.25117522,  0.28229505, -0.05980197,
        -0.73671365, -0.55260205,  0.08691197,  0.08244086, -1.3579206 ,
         0.3993887 , -0.5992716 ,  0.73926276,  0.64969933, -0.32441717,
         0.33525264, -1.1349468 ,  0.1502967 ,  0.8454977 , -1.1183991 ,
        -0.8927187 ,  0.9534588 ,  0.22556058, -0.66705996, -0.65366906,
         0.46913755,  0.5412825 , -0.20910901,  0.31495017, -0.91272557,
         0.02957173,  0.1557533 , -0.6307922 ,  0.43090525, -0.46083456,
        -1.018203  , -0.44839182, -0.35512343, -0.41938427,  0.10584122],
       [ 0.42595312,  0.41425827,  0.05380295,  0.65693736,  0.02124309,
        -0.34377313,  0.27501962, -0.29560134, -0.8899341 ,  0.4253666 ,
        -0.03075071,  0.15959139,  0.05135492,  0.41358554, -0.189

In [0]:
model2 = tf.keras.Sequential()
model2.add(hub_layer)
model2.add(tf.keras.layers.Dense(16, activation='relu'))
model2.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [0]:
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [10]:
model2.fit(train_data.shuffle(10000).batch(512), epochs=20,
           validation_data=validation_data.batch(512), verbose=1)

Epoch 1/20


W0803 19:22:49.367264 139630417131392 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


30/30 [==============================] - 23s 764ms/step - loss: 0.6631 - accuracy: 0.6152 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 22s 730ms/step - loss: 0.5476 - accuracy: 0.7698 - val_loss: 0.5063 - val_accuracy: 0.7931
Epoch 3/20
30/30 [==============================] - 22s 730ms/step - loss: 0.4161 - accuracy: 0.8487 - val_loss: 0.4032 - val_accuracy: 0.8414
Epoch 4/20
30/30 [==============================] - 22s 732ms/step - loss: 0.3008 - accuracy: 0.8981 - val_loss: 0.3417 - val_accuracy: 0.8603
Epoch 5/20
30/30 [==============================] - 22s 731ms/step - loss: 0.2161 - accuracy: 0.9289 - val_loss: 0.3116 - val_accuracy: 0.8701
Epoch 6/20
30/30 [==============================] - 22s 731ms/step - loss: 0.1557 - accuracy: 0.9544 - val_loss: 0.3001 - val_accuracy: 0.8752
Epoch 7/20
30/30 [==============================] - 22s 726ms/step - loss: 0.1115 - accuracy: 0.9717 - val_loss: 0.2995 - val_accuracy: 0.8755
Ep

In [12]:
results = model2.evaluate(test_data.batch(512), verbose=0)
for name, value in zip(model2.metrics_names, results):
  print('%s: %.3f' % (name, value))

loss: 0.481
accuracy: 0.847
